In [1]:
from utility import label_yolo_to_csv
import os
import pandas as pd
import glob
import numpy as np
import shutil
from PIL import Image
from tqdm.auto import tqdm
import random

YOLO_LABELS_B= r"C:\project\Computer_Vision\term_paper_2023\datasets\Anti_UAV_yolo_1024\train\labels"
YOLO_IMAGES_B= r"C:\project\Computer_Vision\term_paper_2023\datasets\Anti_UAV_yolo_1024\train\images"
SAVE_PATH =  r"C:\project\Computer_Vision\term_paper_2023\AttentionGAN_dataset_100"
YOLO_LABELS_A= r"C:\project\Computer_Vision\term_paper_2023\generate_Phantom4_CGI_background_antiUAV_3\labels"
YOLO_IMAGES_A=r"C:\project\Computer_Vision\term_paper_2023\generate_Phantom4_CGI_background_antiUAV_3\images"


In [2]:
def crop_image(path_yolo_labels, path_images, path_dataset_output, domain):

    path_images_domain = os.path.join(path_dataset_output, domain)
    if os.path.exists(path_images_domain): shutil.rmtree(path_images_domain)
    os.makedirs(path_images_domain)

    df = label_yolo_to_csv(path_yolo_labels)
    df['area'] = df.apply(lambda x : x.w*x.h, axis=1)
    df['path_label'] = df['id_image'].apply(lambda id : os.path.join(path_yolo_labels,f'{id}.txt'))
    df['path_image'] = df['id_image'].apply(lambda id : os.path.join(path_images,f'{id}.jpg'))

    image_size=1024
    indent = 50
    x0 = []
    y0 = []
    x1 = []
    y1 = []
    id_image = []
    path_images = []

    for i,row in tqdm(df.iterrows()):
        drone = Image.open(row['path_image']).convert("RGB")
        xc = int(row['xc']*image_size)
        yc = int(row['yc']*image_size)
        w = int(row['w']*image_size)
        h = int(row['h']*image_size)
        area = row['area']
        min_area = 0.0007
        max_area = 0.001

        x0_ = xc - indent
        y0_ = yc - indent
        x1_ = xc + indent
        y1_ = yc + indent
        print(area,min_area,max_area)
        if x0_<0 or y0_<0 or x1_>image_size or y1_>image_size:
            continue
        if w > indent*2  or h  > indent*2:
            continue
        if area < min_area  or area > max_area:
            continue

        x0.append(x0_)
        y0.append(y0_)
        x1.append(x1_)
        y1.append(y1_)
        id_image.append(row['id_image'])

        # print(x0,y0,x1,y1,indent)
        drone = drone.crop((x0_,
                            y0_,
                            x1_,
                            y1_))
        path_image = os.path.join(path_images_domain,f"{row['id_image']}.jpg")
        drone.save(path_image)

    df  = pd.DataFrame({
        'x0':x0,
        'y0':y0,
        'x1':x1,
        'y1':y1,
        'id_image':id_image
    })
    df.to_csv(os.path.join(path_dataset_output, f"markup_{domain}.csv"))

    df = df.sample(frac=1)
    df_train = df
    df_test = df

    path_train = os.path.join(path_dataset_output, f"train{domain}")
    path_test = os.path.join(path_dataset_output, f"test{domain}")

    if os.path.exists(path_train): shutil.rmtree(path_train)
    os.makedirs(path_train)
    if os.path.exists(path_test): shutil.rmtree(path_test)
    os.makedirs(path_test)

    for i, row in df_train.iterrows():
        path = os.path.join(path_images_domain,f"{row['id_image']}.jpg")
        shutil.copyfile(path, os.path.join(path_train,f"{row['id_image']}.jpg"))

    for i, row in df_test.iterrows():
        path = os.path.join(path_images_domain,f"{row['id_image']}.jpg")
        shutil.copyfile(path, os.path.join(path_test,f"{row['id_image']}.jpg"))

    shutil.rmtree(path_images_domain)

In [3]:
# if os.path.exists(SAVE_PATH): shutil.rmtree(SAVE_PATH)
# os.makedirs(SAVE_PATH)
crop_image(YOLO_LABELS_B, YOLO_IMAGES_B, SAVE_PATH, 'B')
crop_image(YOLO_LABELS_A, YOLO_IMAGES_A, SAVE_PATH, 'A')


0it [00:00, ?it/s]

0.049646954986760805 0.0005 0.001
0.28382608695652173 0.0005 0.001
0.3992172211350294 0.0005 0.001
0.36647229621097593 0.0005 0.001
0.29815002759824083 0.0005 0.001
0.042953643067086066 0.0005 0.001
0.28941044127422405 0.0005 0.001
0.031409657320872274 0.0005 0.001
0.020722748815165875 0.0005 0.001
0.33626174496644295 0.0005 0.001
0.22535107489597783 0.0005 0.001
0.2519426180514046 0.0005 0.001
0.29061904761904767 0.0005 0.001
0.3620569673409641 0.0005 0.001
0.2889348025711662 0.0005 0.001
0.03599971200230398 0.0005 0.001
0.08833333333333333 0.0005 0.001
0.04894296784718917 0.0005 0.001
0.030522666666666667 0.0005 0.001
0.03927886056971514 0.0005 0.001
0.005772811918063314 0.0005 0.001
0.012933995447961928 0.0005 0.001
0.21965121722846442 0.0005 0.001
0.2518159806295399 0.0005 0.001
0.044114173228346466 0.0005 0.001
0.31295873305923555 0.0005 0.001
0.3277028286760484 0.0005 0.001
0.25416015625 0.0005 0.001
0.15116472545757073 0.0005 0.001
0.054929577464788736 0.0005 0.001
0.04653266823

0it [00:00, ?it/s]

0.0006361007690429688 0.0005 0.001
0.001251220703125 0.0005 0.001
0.0004673004150390625 0.0005 0.001
0.00112152099609375 0.0005 0.001
0.0006647109985351562 0.0005 0.001
0.000457763671875 0.0005 0.001
0.0010089874267578125 0.0005 0.001
0.0003814697265625 0.0005 0.001
0.000881195068359375 0.0005 0.001
0.0006866455078125 0.0005 0.001
0.0006952285766601562 0.0005 0.001
0.001094818115234375 0.0005 0.001
0.0006084442138671875 0.0005 0.001
0.0010509490966796875 0.0005 0.001
0.000843048095703125 0.0005 0.001
0.0012159347534179688 0.0005 0.001
0.0007677078247070312 0.0005 0.001
0.000579833984375 0.0005 0.001
0.0009756088256835938 0.0005 0.001
0.0010814666748046875 0.0005 0.001
0.0011920928955078125 0.0005 0.001
0.0003986358642578125 0.0005 0.001
0.00054931640625 0.0005 0.001
0.0005321502685546875 0.0005 0.001
0.0005245208740234375 0.0005 0.001
0.000400543212890625 0.0005 0.001
0.00080108642578125 0.0005 0.001
0.0008296966552734375 0.0005 0.001
0.000652313232421875 0.0005 0.001
0.000858306884765